In [1]:
import perceval as pcvl
import numpy as np
import matplotlib.pyplot as plt
import itertools
from Functions import Gaussian, Schmidt, Exponential, Lowdin, Lrz, Overlap, modified_Schmidt
import matplotlib.pyplot as plt
import scipy as sc
from tabulate import tabulate

In [ ]:


new_base,coeffsMGS = modified_Schmidt(waves,x)
C2MGS = coeffsMGS[1,1]